In [459]:
import tensorflow as tf
import numpy as np
import cv2
import input_data

In [460]:
img_height=480
img_width=640
#Network Parameters
n_input=img_height*img_width*3
learning_rate=1e-8
training_iters=10000
batch_size=3
display_step=8
dropout=0.8
epoch=3


tf.reset_default_graph()
#tf graph input
xx=tf.placeholder(tf.float32,[1,img_height*img_width*3])
yy=tf.placeholder(tf.float32,[None,img_height,img_width,16])
keep_prob=tf.placeholder(tf.float32)

In [462]:
def conv2d(img,w,b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def max_pool(img,k):
    return tf.nn.max_pool(img,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
def conv2d_transpose(img,w,b,outputShape):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d_transpose(img,w,output_shape=outputShape,strides=[1,1,1,1],padding='SAME'),b))
def conv2d_output(img,w,b):
    return tf.nn.sigmoid(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def bottleneck(img,w1,w2):
    tmp_bn_1=tf.nn.relu(tf.nn.conv2d(img,w1,strides=[1,1,1,1],padding='SAME'))
    tmp_bn_2=tf.nn.conv2d(tmp_bn_1,w2,strides=[1,1,1,1],padding='SAME')
    return (img+tmp_bn_2)

In [463]:
wc1=tf.Variable(tf.random_normal([7,7,3,64]),name="wc1")
bc1=tf.Variable(tf.random_normal([64]),name="bc1")

wc2=tf.Variable(tf.random_normal([5,5,256,64]),name="wc2")
bc2=tf.Variable(tf.random_normal([64]),name="bc2")

wc3=tf.Variable(tf.random_normal([5,5,64,16]),name="wc3")
bc3=tf.Variable(tf.random_normal([16]),name="bc3")

wc4=tf.Variable(tf.random_normal([1,1,16,16]),name="wc4")
bc4=tf.Variable(tf.random_normal([16]),name="bc4")
#bottleneck
w_bn_1=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_1")
w_bn_2=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_2")

w_bn_3=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_3")
w_bn_4=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_4")

w_bn_5=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_5")
w_bn_6=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_6")

w_bn_7=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_7")
w_bn_8=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_8")

w_bn_9=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_9")
w_bn_10=tf.Variable(tf.random_normal([5,5,64,64]),name="w_bn_10")

#branches
w_br_1=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_1")
w_br_2=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_2")

w_br_3=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_3")
w_br_4=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_4")

w_br_5=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_5")
w_br_6=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_6")

w_br_7=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_7")
w_br_8=tf.Variable(tf.random_normal([5,5,64,64]),name="w_br_8")

#deconcolution
w_dc_1=tf.Variable(tf.random_normal([2,2,64,64]),name="w_dc_1")
b_dc_1=tf.Variable(tf.random_normal([64]),name="b_dc_1")

w_dc_2=tf.Variable(tf.random_normal([2,2,64,64]),name="w_dc_2")
b_dc_2=tf.Variable(tf.random_normal([64]),name="b_dc_2")

w_dc_3=tf.Variable(tf.random_normal([2,2,64,64]),name="w_dc_3")
b_dc_3=tf.Variable(tf.random_normal([64]),name="b_dc_3")

w_dc_4=tf.Variable(tf.random_normal([2,2,64,64]),name="w_dc_4")
b_dc_4=tf.Variable(tf.random_normal([64]),name="b_dc_4")

w_dc_5=tf.Variable(tf.random_normal([2,2,256,64]),name="w_dc_5")
b_dc_5=tf.Variable(tf.random_normal([256]),name="b_dc_5")

w_dc_6=tf.Variable(tf.random_normal([2,2,64,64]),name="w_dc_6")
b_dc_6=tf.Variable(tf.random_normal([64]),name="b_dc_6")



In [464]:
_X=tf.reshape(xx,shape=[-1,img_height,img_width,3])
conv1=conv2d(_X,wc1,bc1)
conv1_pooling=max_pool(conv1,k=2)
conv1_pooling=tf.nn.dropout(conv1_pooling,keep_prob)


bn1=bottleneck(conv1_pooling,w_bn_1,w_bn_2)
bn1=tf.nn.dropout(bn1,keep_prob)

bn1_pooling=max_pool(bn1,k=2)
bn2=bottleneck(bn1_pooling,w_bn_3,w_bn_4)
bn2=tf.nn.dropout(bn2,keep_prob)

bn2_pooling=max_pool(bn2,k=2)
bn3=bottleneck(bn2_pooling,w_bn_5,w_bn_6)
bn3=tf.nn.dropout(bn3,keep_prob)

bn3_pooling=max_pool(bn3,k=2)
bn4=bottleneck(bn3_pooling,w_bn_7,w_bn_8)
bn4=tf.nn.dropout(bn4,keep_prob)

bn4_pooling=max_pool(bn4,k=2)
bn5=bottleneck(bn4_pooling,w_bn_9,w_bn_10)
bn5=tf.nn.dropout(bn5,keep_prob)

bran1=bottleneck(bn5,w_br_1,w_br_2)

tmpShape=bran1.get_shape().as_list()
tmpShape[0]=batch_size
dc1=conv2d_transpose(bn5,w_dc_1,b_dc_1,outputShape=tmpShape)

add1=dc1+bran1
add1=tf.nn.dropout(add1,keep_prob)


bran2=bottleneck(bn4,w_br_3,w_br_4)

add1=tf.image.resize_bilinear(add1,bn4.get_shape().as_list()[1:3])
tmpShape=bran2.get_shape().as_list()
tmpShape[0]=batch_size
dc2=conv2d_transpose(add1,w_dc_2,b_dc_2,outputShape=tmpShape)

add2=dc2+bran2
add2=tf.nn.dropout(add2,keep_prob)


bran3=bottleneck(bn3,w_br_5,w_br_6)

add2=tf.image.resize_bilinear(add2,bn3.get_shape().as_list()[1:3])
tmpShape=bran3.get_shape().as_list()
tmpShape[0]=batch_size
dc3=conv2d_transpose(add2,w_dc_3,b_dc_3,outputShape=tmpShape)

add3=dc3+bran3
add3=tf.nn.dropout(add3,keep_prob)


bran4=bottleneck(bn2,w_br_7,w_br_8)

add3=tf.image.resize_bilinear(add3,bn2.get_shape().as_list()[1:3])
tmpShape=bran4.get_shape().as_list()
tmpShape[0]=batch_size
dc4=conv2d_transpose(add3,w_dc_4,b_dc_4,outputShape=tmpShape)

add4=dc4+bran4
add4=tf.nn.dropout(add4,keep_prob)


add4=tf.image.resize_bilinear(add4,bn1.get_shape().as_list()[1:3])
tmpShape=bn1.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=256
dc5=conv2d_transpose(add4,w_dc_5,b_dc_5,outputShape=tmpShape)

conv2=conv2d(dc5,wc2,bc2)
conv2=tf.nn.dropout(conv2,keep_prob)

conv2=tf.image.resize_bilinear(conv2,_X.get_shape().as_list()[1:3])
tmpShape=_X.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=64
dc6=conv2d_transpose(conv2,w_dc_6,b_dc_6,outputShape=tmpShape)

conv3=conv2d(dc6,wc3,bc3)
conv3=tf.nn.dropout(conv3,keep_prob)

conv4=conv2d(conv3,wc4,bc4)

cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=conv_t2,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [465]:
saver = tf.train.Saver({"wc1":wc1,"bc1":bc1,"wc2":wc2,"bc2":bc2,"wc3":wc3,"bc3":bc3,"wc4":wc4,"bc4":bc4,
                        "w_bn_1":w_bn_1,"w_bn_2":w_bn_2,"w_bn_3":w_bn_3,"w_bn_4":w_bn_4,"w_bn_5":w_bn_5,"w_bn_6":w_bn_6,
                        "w_bn_7":w_bn_7,"w_bn_8":w_bn_8,"w_bn_9":w_bn_9,"w_bn_10":w_bn_10,
                        "w_br_1":w_br_1,"w_br_2":w_br_2,"w_br_3":w_br_3,"w_br_4":w_br_4,"w_br_5":w_br_5,"w_br_6":w_br_6,
                        "w_br_7":w_br_1,"w_br_8":w_br_8,
                        "w_dc_1":w_dc_1,"w_dc_2":w_dc_2,"w_dc_3":w_dc_3,"w_dc_4":w_dc_4,"w_dc_5":w_dc_5,"w_dc_6":w_dc_6
                       })
init=tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [466]:
with tf.Session(config=config) as sess:
    sess.run(init)
    saver = tf.train.Saver()
    #saver.restore(sess,"model/model.ckpt")
    for i in range(0,epoch):
        print("epoch:"+str(i)+"       !!!!!!!!!!!!")
        step=0
        now_at=0
        while step*batch_size<training_iters:
            X,Y=input_data.next_batch(
                    img_dir_path='mpii_human_pose_v1\\output_images\\',
                    index_path='train_data\\new_data.json',
                    img_height=img_height,img_width=img_width,
                    batch_size=batch_size,now_at=now_at)
            sess.run(optimizer,feed_dict = {x:xx,y:yy,keep_prob:dropout})
            loss=sess.run(cost,feed_dict = {x:xx,y:yy,keep_prob:1.})
            if step%display_step==0:
                print("Iter "+str(step*batch_size)+", Minibatch Loss="+"{:.6f}".format(loss))
            step+=1
            now_at+=batch_size
    save_path = saver.save(sess, "model/model_complex.ckpt")
print("done!")

(1, 480, 640, 16)
